In [ ]:
from ultralytics import YOLO
import os
import yaml
import boto3

In [ ]:
! tar -xzf dataset-full.tar.gz

In [ ]:
EPOCHS=int(os.environ.get("epochs", 1))
IMGSZ=640
DATA_CONFIG_PATH = "utils/data.yaml"
LABEL_NAME = os.environ.get("label_name", "mylabel")

EXPORT_BUCKET = "model-zoo"
MODEL_PATH = "models/model.onnx"

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.environ.get("AWS_S3_BUCKET")
AWS_S3_ENDPOINT = os.environ.get("AWS_S3_ENDPOINT")

In [ ]:
with open(DATA_CONFIG_PATH, 'r') as f:
    data_config = yaml.safe_load(f)
data_config['names'][0] = LABEL_NAME
with open(DATA_CONFIG_PATH, 'w') as f:
    data_config = yaml.dump(data_config, f)

In [ ]:
! ls

In [ ]:
model = YOLO('yolov8n.pt')
results = model.train(data=DATA_CONFIG_PATH, epochs=EPOCHS, imgsz=IMGSZ)

In [ ]:
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
model.export(format="onnx")

In [ ]:
model_path = "runs/detect/train/weights/best.onnx"

In [ ]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)
client = session.client("s3", endpoint_url=AWS_S3_ENDPOINT)

In [ ]:
client.upload_file(model_path, EXPORT_BUCKET, MODEL_PATH)